<a href="https://colab.research.google.com/github/shubhamchourasia/LangChainBasedInvoiceReader/blob/main/Pinned_External_Financial_Advisor_Sunday_16th_Nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Installing packages

In [ ]:
!pip install -q \
requests~=2.32.4 \
langchain~=0.3.27 \
langchain-core~=0.3.79 \
langchain-community~=0.3.31 \
langchain-google-genai~=2.0.10 \
langchain-huggingface~=0.3.1 \
langgraph~=1.0.1 \
SQLAlchemy~=2.0.44 \
chromadb==1.3.0 \
faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.2 MB/s eta 0:00:00
  

In [ ]:
!pip show \
requests \
langchain \
langchain-core \
langchain-community \
langchain-google-genai \
langchain-huggingface \
langgraph \
SQLAlchemy \
chromadb \
faker

Name: requests
Version: 2.32.5
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: certifi, charset_normalizer, idna, urllib3
Required-by: bigframes, blosc2, CacheControl, community, datasets, diffusers, earthengine-api, fastai, folium, gcsfs, gdown, geocoder, google-adk, google-api-core, google-cloud-bigquery, google-cloud-storage, google-colab, google-genai, googledrivedownloader, huggingface-hub, jupyter_kernel_gateway, kaggle, kagglehub, keyrings.google-artifactregistry-auth, kubernetes, langchain, langchain-community, langsmith, libpysal, moviepy, music21, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, pandas-datareader, panel, pooch, posthog, requests-oauthlib, requests-toolbelt, spacy, spanner-graph-notebook, Sphinx, tensorflow, tensorflow-datasets, tiktoken, torchdata, transformers, tsfresh, 

## Add your Gemini's API Key in secrets

In [ ]:
# Add your GEMINI_API_KEY in Secrets
from google.colab import userdata
import os

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

## Pull the helper file which already has database declarations so we don't have to dive into that

In [ ]:
!wget https://raw.githubusercontent.com/NavSrijan/langchain_setup/refs/heads/master/db_setup.py

## Setting up the database

In [ ]:
from db_setup import setup, get_db, User, Transaction, Goal, populate_dummy_data, TransactionType
from sqlalchemy import select, func

setup = setup()
db = get_db()
populate_dummy_data(db)


✓ Database initialized at financial_advisor.db
Populating database with 5 users, 40 transactions, and 5 goals...
✓ Dummy data populated successfully.


In [ ]:
session = db.get_session()
user = session.query(User).filter(User.id == 3).first()
print(f"\nUser ID: {user.id}")
print(f"Username: {user.username}")
for x in user.transactions:
    print(f"Description: {x.description}, Amount: {x.amount}, Date: {x.transaction_date}")


User ID: 3
Username: elizabethpittman
Description: Electronics store, Amount: 201.7100000000, Date: 2025-03-07 08:05:55.946767
Description: Streaming service, Amount: 224.0800000000, Date: 2024-12-16 08:05:55.957656
Description: Bookstore purchase, Amount: 345.1700000000, Date: 2025-10-22 08:05:55.965822
Description: Crypto sale, Amount: 2737.9100000000, Date: 2025-05-03 08:05:55.990914
Description: Local supermarket, Amount: 388.1000000000, Date: 2024-11-30 08:05:56.007381
Description: Performance bonus, Amount: 1464.4100000000, Date: 2025-11-15 08:05:56.057314
Description: Dentist visit, Amount: 141.1700000000, Date: 2025-06-19 08:05:56.091119
Description: Bi-weekly pay, Amount: 5270.8800000000, Date: 2025-01-03 08:05:56.163185
Description: Monthly rent payment, Amount: 183.0300000000, Date: 2025-07-28 08:05:56.221937


In [ ]:
# print(user.transactions)

# Basics of Langchain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI


LangChain helps developers build applications powered by large language models (LLMs). It connects LLMs to external data and tools, enabling more complex and useful AI solutions.


## RAG
RAG combines:

    Retriever → Fetches relevant documents from a vector store.
    LLM → Generates context-aware responses using query + retrieved docs.

Today, we will:

    Store documents with Gemini Embeddings
    Retrieve docs from Chroma
    Use Gemini LLM for answering questions


## Tool calling

In [ ]:
from langchain_core.tools import BaseTool




In [ ]:
from langchain.agents import create_react_agent, AgentType, AgentExecutor, initialize_agent
from langchain_core.prompts import ChatPromptTemplate



In [ ]:
from langchain_core.tools import BaseTool





> Entering new AgentExecutor chain...
Action: multiplier
Action Input: {"a": 128, "b": 2}{'a': 128, 'b': 2}

Observation: The product of 128 and 2 is 256
Thought:Action: divider
Action Input: {"a": 256, "b": 4}
Observation: The division of 256 and 4 is 64.0
Thought:Action: adder
Action Input: {"a": 64, "b": 10}
Observation: The sum of 64 and 10 is 74
Thought:I now know the final answer
Final Answer: 74

> Finished chain.
74


## VectorDB

In [ ]:
from langchain.schema import Document

docs = [
    Document(
        page_content="Virat Kohli is one of the most successful batsmen in IPL history and has captained RCB.",
        metadata={"team": "Royal Challengers Bangalore"}
    ),
    Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, winning five titles with Mumbai Indians.",
        metadata={"team": "Mumbai Indians"}
    ),
    Document(
        page_content="MS Dhoni has led Chennai Super Kings to multiple IPL titles and is known as Captain Cool.",
        metadata={"team": "Chennai Super Kings"}
    ),
    Document(
        page_content="Jasprit Bumrah is a leading fast bowler for Mumbai Indians, famous for his yorkers.",
        metadata={"team": "Mumbai Indians"}
    ),
    Document(
        page_content="Ravindra Jadeja is an all-rounder for Chennai Super Kings, contributing with bat, ball, and fielding.",
        metadata={"team": "Chennai Super Kings"}
    )
]


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=GEMINI_API_KEY)

# Create Chroma vector store


# Add documents


/tmp/ipython-input-97405778.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


['e63798c6-c9b9-4893-ba8e-1009bf3e8b25',
 'f2501f81-1f77-4085-8d59-e4edfbc39462',
 'eb49f981-dbbf-46bf-ba0c-129cd4a6e47b',
 'a5d95c7a-9a56-444c-bebf-865ba0579920',
 'b4eb7a5c-b78a-4f22-baf2-beee240e3ad1']

In [ ]:
# Initialize Retriever

In [ ]:
from langchain.chains import RetrievalQA

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     return_source_documents=True
# )

In [ ]:
# Query 1
query = "Who is the most successful IPL captain?"
response = qa_chain.invoke({"query": query})

print("Query:", query)
print("Answer:", response["result"])
print("\nSources:", [doc.metadata for doc in response["source_documents"]])



Query: Who is the most successful IPL captain?
Answer: Rohit Sharma is the most successful captain in IPL history, winning five titles with Mumbai Indians.

Sources: [{'team': 'Mumbai Indians'}, {'team': 'Royal Challengers Bangalore'}]

Query: Which bowler is famous for yorkers?
Answer: Jasprit Bumrah is famous for his yorkers.

Sources: [{'team': 'Mumbai Indians'}, {'team': 'Chennai Super Kings'}]


# Financial Advisor

We'll be creating three tools here as a demonstration:
1. List Categories
2. Sum of expense for the last X days
3. SQL Query

### List Categories

In [ ]:
# Defining a user


In [ ]:
from langchain_core.tools import BaseTool
from sqlalchemy import select, func
import json

# list categories


In [ ]:
from datetime import timedelta, datetime

# Sum of last X days


In [ ]:
from sqlalchemy import text

# SQL Tool


In [ ]:
# Initialize Agent
